# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

In [58]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [59]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


**Описание данных** <br>
- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [60]:
# Проверим тип данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Изменим тип столбца messages на int

In [61]:
df['messages']=df['messages'].astype("int")

In [62]:
#Проверим дубликаты
df.duplicated().sum()

0

**Вывод:**
- Пропущенных значений нет
- Типы столбцов в порядке
- Дубликатов нет

## Разбейте данные на выборки

*Разобъем выборки на train(60%) и val+test(40%)*

In [63]:
df_train, df_test = train_test_split(df, test_size=0.4, random_state=12345)
features = df_test.drop('is_ultra', axis=1)
target = df_test['is_ultra']

*Разобъем на val(20%) и test(20%)*

In [64]:
features_valid, features_test, target_valid, target_test = train_test_split(features, target,
                                                                            test_size=0.5, random_state=12345)

In [65]:
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

**Напоминалка**
- `features_train`, `target_train` - обучающая выборка
- `features_valid`, `target_valid` - валидационная выборка
- `features_test`, `target_test` - тестовая выборка

## Исследуйте модели

**Логистическая регрессия**

In [66]:
model = LogisticRegression(sortedlver='lbfgs')
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid,predictions)
print("Качество:", accuracy)

Качество: 0.7107309486780715


**Случайный лес**

In [67]:
best_accuracy = 0
best_depth = 0
for depth in range(1,10):
    for est in range(5,45,5):
        for sample in range(2,5):
            model = RandomForestClassifier(max_depth=depth, n_estimators=est, 
                                           min_samples_leaf=sample, random_state=1515)
            model.fit(features_train, target_train)
            predictions = model.predict(features_valid)
            accuracy = accuracy_score(target_valid,predictions)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, "    Количество деревьев:",
      best_est, "   мин. объектов в узле:", best_sample, "  Качество:", best_accuracy)

Глубина дерева: 7     Количество деревьев: 25    мин. объектов в узле: 3   Качество: 0.8055987558320373


**Дерево решений**

In [68]:
best_accuracy = 0
best_depth = 0
for depth in range(1,15):
    for sample in range(2,5):
        model = DecisionTreeClassifier(max_depth=depth, min_samples_leaf=sample, random_state=1515)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        accuracy = accuracy_score(target_valid,predictions)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = depth
            best_sample = sample
print("Глубина дерева:", best_depth,"   мин. объектов в узле:", best_sample, "    Качество:", best_accuracy)

Глубина дерева: 6    мин. объектов в узле: 4     Качество: 0.7869362363919129


**Вывод:**<br>
Были исследованы три модели: логистическая регрессия, случайный лес, деревья решений. Предварительно, наилучшее качество (0.805) без проверки достигается при использовании Случайного леса с гиперпараметрами: глубина дерева 7, количество деревьев 25, мин. объектов в узле 3<br>
Результат необходимо проверить, приступим

## Проверьте модель на тестовой выборке

**Логистическая регрессия**

In [69]:
model = LogisticRegression(solver='lbfgs')
model.fit(features_train, target_train)
predictions = model.predict(features_train)
accuracy = accuracy_score(target_train,predictions)
print("Качество:", accuracy)

Качество: 0.7131742738589212


**Случайный лес**

In [70]:
model = RandomForestClassifier(max_depth=7, n_estimators=25, min_samples_leaf=3, random_state=1515)
model.fit(features_train, target_train)
predictions = model.predict(features_train)
accuracy = accuracy_score(target_train,predictions)
print("Качество:", accuracy)

Качество: 0.8475103734439834


**Дерево решений**

In [71]:
model = DecisionTreeClassifier(max_depth=6,min_samples_leaf=4, random_state=1515)
model.fit(features_train, target_train)
predictions = model.predict(features_train)
accuracy = accuracy_score(target_train,predictions)
print("Качество:", accuracy)

Качество: 0.8340248962655602


**Вывод:**<br>
После проверки наших моделей лидером так и осталась модель Случайного леса с метрикой качества 0.847. Дерево решений тоже показало хороший результат с метрикой качества 0.834. Переобучение у моделей примерно одинаковое. Что касается логистической регрессии, то ее метрика качества приблизительно на том же значении - 0.713. <br>

## (бонус) Проверьте модели на адекватность

In [72]:
dmb = DummyClassifier(strategy="stratified" ,random_state=1515)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество наивного прогноза :", acc)
print("Качество нашего прогноза :", accuracy)

Качество наивного прогноза : 0.5800933125972006
Качество нашего прогноза : 0.8340248962655602


In [73]:
dmb = DummyClassifier(strategy='most_frequent', random_state=1515)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество наивного прогноза :", acc)
print("Качество нашего прогноза :", accuracy)

Качество наивного прогноза : 0.6842923794712286
Качество нашего прогноза : 0.8340248962655602


In [74]:
dmb = DummyClassifier(strategy='uniform', random_state=1515)
dmb.fit(features_train, target_train)
acc = dmb.score(features_test, target_test)
print("Качество наивного прогноза :", acc)
print("Качество нашего прогноза :", accuracy)

Качество наивного прогноза : 0.5147744945567652
Качество нашего прогноза : 0.8340248962655602


# Вывод

**Были исследованы три модели: логистическая регрессия, случайный лес, дерево решений. После проверки наших моделей лидером является модель Случайного леса с метрикой качества 0.847. Дерево решений тоже показало хороший результат с метрикой качества 0.834. Переобучение у моделей примерно одинаковое. Что касается логистической регрессии, то ее метрика качества даже после проверки остается достаточно низкой - 0.713.<br> Наилучшее качество при обучении модели (0.805) достигается при использовании Случайного леса с гиперпараметрами: глубина дерева 7, количество деревьев 25, мин. объектов в узле 3.**

In [75]:
features_full = pd.concat([features_train,features_valid])
target_full = pd.concat([target_train,target_valid])
model = RandomForestClassifier(max_depth=7, n_estimators=25, 
                                           min_samples_leaf=3, random_state=1515)
model.fit(features_full, target_full)
predictions = model.predict(features_test)
accuracy = accuracy_score(target_test,predictions)
print("Качество:", accuracy)

Качество: 0.7962674961119751


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
